In [45]:
# ==============================================================================
# Notebook 4: Simulação de Novos Clientes e Conclusão
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. Configuração e Carregamento de Bibliotecas e Artefatos
# ------------------------------------------------------------------------------
import pandas as pd
import numpy as np
import joblib
import os
import json

In [46]:
# --- Definindo os caminhos conforme a estrutura de diretórios ---
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

FINAL_MODEL_PATH = os.path.join(ROOT_DIR, 'models', 'final_model', 'random_forest_final.pkl')
PREPROCESSING_PATH = os.path.join(ROOT_DIR, 'models', 'preprocessing')
REPORTS_PATH = os.path.join(ROOT_DIR, 'reports')


In [47]:
# Carregando o modelo final e os objetos de pré-processamento
print("Carregando modelo final e objetos de pré-processamento...")
try:
    final_model = joblib.load(FINAL_MODEL_PATH)
    scaler = joblib.load(os.path.join(PREPROCESSING_PATH, 'standard_scaler.pkl'))
    label_encoder_churn = joblib.load(os.path.join(PREPROCESSING_PATH, 'label_encoder_churn.pkl'))
    print("✅ Artefatos carregados com sucesso.")
except FileNotFoundError as e:
    print(f"❌ Erro ao carregar artefatos: {e}. Certifique-se de ter executado os notebooks anteriores.")
    final_model, scaler, label_encoder_churn = None, None, None


Carregando modelo final e objetos de pré-processamento...
✅ Artefatos carregados com sucesso.


In [48]:
# ------------------------------------------------------------------------------
# 2. Simulação de Novos Dados
# ------------------------------------------------------------------------------
if final_model is not None:
    print("\nSimulando novos dados de clientes...")
    
    # Criando um DataFrame com novos dados simulados
    new_customers_data = {
        'gender': ['Female', 'Male', 'Female', 'Male'],
        'SeniorCitizen': [0, 1, 0, 1],
        'Partner': ['Yes', 'No', 'No', 'Yes'],
        'Dependents': ['Yes', 'No', 'No', 'No'],
        'tenure': [12, 1, 36, 72],
        'PhoneService': ['Yes', 'Yes', 'No', 'Yes'],
        'MultipleLines': ['Yes', 'No', 'No', 'Yes'],
        'InternetService': ['Fiber optic', 'DSL', 'DSL', 'Fiber optic'],
        'OnlineSecurity': ['No', 'No', 'No', 'Yes'],
        'OnlineBackup': ['Yes', 'No', 'No', 'Yes'],
        'DeviceProtection': ['No', 'No', 'No', 'Yes'],
        'TechSupport': ['No', 'No', 'No', 'Yes'],
        'StreamingTV': ['Yes', 'No', 'No', 'Yes'],
        'StreamingMovies': ['Yes', 'No', 'No', 'Yes'],
        'Contract': ['Month-to-month', 'Month-to-month', 'One year', 'Two year'],
        'PaperlessBilling': ['Yes', 'No', 'No', 'Yes'],
        'PaymentMethod': ['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'],
        'MonthlyCharges': [89.0, 45.0, 30.0, 110.0],
        'TotalCharges': [1068.0, 45.0, 1080.0, 7920.0]
    }
    
    new_customers_df = pd.DataFrame(new_customers_data)


Simulando novos dados de clientes...


In [49]:
# ------------------------------------------------------------------------------
# 3. Pré-processamento dos Novos Dados (utilizando os artefatos salvos)
# ------------------------------------------------------------------------------
print("Pré-processando os novos dados...")
    
# Mapeamento binário
new_customers_df['gender'] = new_customers_df['gender'].map({'Female': 0, 'Male': 1})
new_customers_df['Partner'] = new_customers_df['Partner'].map({'No': 0, 'Yes': 1})
new_customers_df['Dependents'] = new_customers_df['Dependents'].map({'No': 0, 'Yes': 1})
new_customers_df['PhoneService'] = new_customers_df['PhoneService'].map({'No': 0, 'Yes': 1})
new_customers_df['PaperlessBilling'] = new_customers_df['PaperlessBilling'].map({'No': 0, 'Yes': 1})

Pré-processando os novos dados...


In [50]:
 # One-Hot Encoding
new_customers_df = pd.get_dummies(new_customers_df, columns=[
    'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
    'PaymentMethod', 'InternetService'
], drop_first=True)

In [51]:
# Escalonamento
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
new_customers_df[numeric_cols] = scaler.transform(new_customers_df[numeric_cols])

In [52]:
# Garantindo que as colunas estejam na mesma ordem que o X_train
# Isso é essencial para a previsão!
# A forma mais robusta seria salvar a lista de colunas do X_train no Notebook 2
# e carregar aqui. Para simplificar, vamos usar uma lista estática.
    
# Criação de um X_test dummy para obter a ordem correta das colunas
# do One-Hot Encoding que o modelo espera
X_train_path = os.path.join(ROOT_DIR, 'data', 'processed', 'split', 'X_train.csv')
X_train_df = pd.read_csv(X_train_path)
    
# Alinhando as colunas
new_customers_aligned = new_customers_df.reindex(columns = X_train_df.columns, fill_value=0)
print("✅ Novos dados pré-processados e alinhados com sucesso.")

✅ Novos dados pré-processados e alinhados com sucesso.


In [53]:
# ------------------------------------------------------------------------------
# 4. Previsão de Churn com o Modelo Final
# ------------------------------------------------------------------------------
print("\nFazendo a previsão de Churn para os novos clientes...")
    
churn_probabilities = final_model.predict_proba(new_customers_aligned)[:, 1]
# Usando o threshold otimizado para a previsão final
# É necessário carregar o threshold do arquivo JSON para isso
try:
    with open(os.path.join(REPORTS_PATH, 'final_model_metrics.json'), 'r') as f:
        metrics = json.load(f)
    optimized_threshold = metrics['best_threshold']
    churn_predictions = (churn_probabilities >= optimized_threshold).astype(int)
except FileNotFoundError:
    print("❌ Erro: Arquivo de métricas do modelo final não encontrado. Usando limiar padrão de 0.5.")
    churn_predictions = final_model.predict(new_customers_aligned)

new_customers_df['Probability of Churn'] = churn_probabilities
new_customers_df['Predicted Churn (Binary)'] = churn_predictions
    
new_customers_df['Predicted Churn'] = label_encoder_churn.inverse_transform(churn_predictions)
    
print("✅ Previsões realizadas com sucesso.")
print("\nResultados da Simulação:")
print(new_customers_df[['Probability of Churn', 'Predicted Churn']].head())


Fazendo a previsão de Churn para os novos clientes...
✅ Previsões realizadas com sucesso.

Resultados da Simulação:
   Probability of Churn Predicted Churn
0                  0.72             Yes
1                  0.73             Yes
2                  0.19              No
3                  0.01              No


In [54]:
# ------------------------------------------------------------------------------
# 5. Conclusão e Comparação de Resultados
# ------------------------------------------------------------------------------
print("\n--- Conclusão do Projeto ---")
    
try:
    base_report = pd.read_csv(os.path.join(REPORTS_PATH, 'comparacao_modelos_base.csv'))
        
    # Carregando as métricas do modelo final do arquivo JSON
    with open(os.path.join(REPORTS_PATH, 'final_model_metrics.json'), 'r') as f:
        final_metrics = json.load(f)
        
    print("\nSumário de Performance dos Modelos:")
    print("Modelo Base (Random Forest) vs. Modelo Final Otimizado:")
        
    base_metrics = base_report[base_report['Modelo'] == 'Random Forest'].iloc[0]
        
    comparison = pd.DataFrame({
        'Métrica': ['Precisão', 'Recall', 'F1-Score'],
        'Modelo Base (RF)': [base_metrics['Precisao'], base_metrics['Recall'], base_metrics['F1-Score']],
        'Modelo Otimizado (RF)': [final_metrics['precision'], final_metrics['recall'], final_metrics['f1-score']]
    })
        
    print(comparison)
        
    feature_importance_df = pd.read_csv(os.path.join(REPORTS_PATH, 'feature_importance.csv'))
    print("\nPrincipais Insights de Negócio (Feature Importance):")
    print(feature_importance_df.head(5))
        
    print("\nRecomendações de Negócio:")
    print("- Clientes com menor tempo de contrato (`tenure`) e com internet de Fibra Óptica (`InternetService_Fiber optic`) têm maior risco de churn.")
    print("- Acompanhar de perto clientes com 'Electronic check' como método de pagamento.")
    print("- Considerar ofertas de retenção para clientes com alta pontuação de risco de churn, especialmente aqueles com as características de maior importância.")
        
except FileNotFoundError as e:
    print(f"\n❌ Erro: Não foi possível carregar os relatórios para a comparação. {e}. Certifique-se de que os notebooks anteriores foram executados e salvaram os arquivos corretamente.")
except Exception as e:
    print(f"\n❌ Erro durante a comparação de modelos: {e}. Verifique o formato dos arquivos de relatório.")

print("\nProcesso do Notebook 4 concluído com sucesso! Refatoração finalizada.")


--- Conclusão do Projeto ---

Sumário de Performance dos Modelos:
Modelo Base (Random Forest) vs. Modelo Final Otimizado:
    Métrica  Modelo Base (RF)  Modelo Otimizado (RF)
0  Precisão          0.629187               0.702133
1    Recall          0.468806               0.754783
2  F1-Score          0.537283               0.702134

Principais Insights de Negócio (Feature Importance):
                          Feature  Importance
0                          tenure    0.175106
1                    TotalCharges    0.173941
2                  MonthlyCharges    0.147675
3     InternetService_Fiber optic    0.053770
4  PaymentMethod_Electronic check    0.050929

Recomendações de Negócio:
- Clientes com menor tempo de contrato (`tenure`) e com internet de Fibra Óptica (`InternetService_Fiber optic`) têm maior risco de churn.
- Acompanhar de perto clientes com 'Electronic check' como método de pagamento.
- Considerar ofertas de retenção para clientes com alta pontuação de risco de churn, espe